# Exercise 1: t-SNE

## Do not start the exercise until you fully understand the submission guidelines.


* The homework assignments are executed automatically. 
* Failure to comply with the following instructions will result in a significant penalty. 
* Appeals regarding your failure to read these instructions will be denied. 
* Kind reminder: the homework assignments contribute 60% of the final grade.


## Read the following instructions carefully:

1. This Jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Write **efficient**, **vectorized** code whenever possible. Some calculations in this exercise may take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deductions.
1. You are responsible for the correctness of your code and should add as many tests as you see fit to this jupyter notebook. Tests will not be graded nor checked.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/).
1. Your code must run without errors. Use at least `numpy` 1.15.4. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Submission includes a zip file that contains this notebook, with your ID as the file name. For example, `hw1_123456789_987654321.zip` if you submitted in pairs and `hw1_123456789.zip` if you submitted the exercise alone. The name of the notebook should follow the same structure.
   
Please use only a **zip** file in your submission.

---
---

## Please sign that you have read and understood the instructions: 

316492776
---
---


In [1]:
# Import necessary libraries
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.random.seed(42)

# Design your algorithm
Make sure to describe the algorithm, its limitations, and describe use-cases.

# Your implementations
You may add new cells, write helper functions or test code as you see fit.
Please use the cell below and include a description of your implementation.
Explain code design consideration, algorithmic choices and any other details you think is relevant to understanding your implementation.
Failing to explain your code will lead to point deductions.

In [4]:
from scipy.spatial.distance import pdist, squareform

In [58]:
class CustomTSNE:
    def __init__(self, perplexity=30.0, n_components=2, n_iter=1000, learning_rate=200.0):
        self.perplexity = perplexity
        self.n_components = n_components
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        # Note: You may add more attributes
        self.sigma = 1  # TODO

    def fit_transform(self, X):
        # Part 1: Implementing t-SNE

        # Step 1: Compute pairwise affinities in the original space with a Gaussian distribution
        # Your code here
        # Implementation is according to slides 61-66
        N = len(X)
        X_dists = pdist(X)
        X_dists = -X_dists**2 / self.sigma
        X_dists = np.exp(X_dists)
        X_dists = squareform(X_dists) # diagonal is 0
        p_denominator = np.sum(X_dists)
        p_ij = X_dists / p_denominator
        p_ij = (p_ij + p_ij.T) / (2*N)

        Y = np.random.random((N, self.n_components))

        for _ in range(self.n_iter):
            Y_dists = pdist(Y)
            Y_dists = 1+Y_dists**2
            Y_dists = Y_dists ** -1
            Y_dists = squareform(Y_dists)
            q_denominator = np.sum(Y_dists) # diagonal is 0
            q_ij = Y_dists / q_denominator

            C = p_ij * np.log(p_ij/q_ij, where=(q_ij != 0) & (p_ij != 0))  # remove i=j cases
            # TODO: remove division by 0 (although it doesnt do anything)
            
            C = np.sum(C)

            Y_diff = Y[:, np.newaxis, :] - Y[np.newaxis, :, :]
            grad_C_yi = 4 * \
                np.sum((p_ij - q_ij)[..., np.newaxis] *
                       Y_diff * Y_dists[..., np.newaxis], axis=1)

            Y += self.learning_rate * grad_C_yi
            # TODO: add early stopping
            # TODO: maybe take only close neighbors
        # Return Y, the 2D representation of the input data
        return Y

    # Part 2: Transformation of New Data Points

    def transform(self, X_original, Y_original, X_new):
        # Implement your method for incorporating new points into the existing t-SNE layout
        # Your code here

        # Return Y_new, the transformed data
        pass

In [59]:
CustomTSNE().fit_transform(np.random.random((100, 200)))

C:\Users\Roey\AppData\Local\Temp\ipykernel_2844\2002700946.py:35: RuntimeWarning: invalid value encountered in divide
  C = p_ij * np.log(p_ij/q_ij, where=(q_ij != 0) & (p_ij != 0))  # remove i=j cases


array([[ 0.59986559,  0.24883919],
       [ 0.64885515,  0.56444187],
       [ 0.343654  ,  0.32338828],
       [ 0.28920073,  0.56410805],
       [ 0.29344076,  0.91622531],
       [ 0.64690825,  0.11863141],
       [ 0.58410881,  0.46122467],
       [ 0.77792654,  0.98653834],
       [ 0.37374789,  0.62191628],
       [ 0.38630591,  0.4926952 ],
       [ 0.67658795,  0.83588504],
       [ 0.24408007,  0.7460703 ],
       [ 0.40490002, -0.01446908],
       [ 0.49342935,  0.33782054],
       [ 0.2068212 ,  0.92744979],
       [ 0.52952402,  0.93146622],
       [ 0.22279258,  0.48856804],
       [ 0.89995392,  0.54934332],
       [ 0.61457612,  0.0596524 ],
       [ 0.42024994,  0.65650785],
       [ 0.39211974,  0.23614835],
       [ 0.27643564,  0.38183213],
       [ 0.33943102,  0.98208229],
       [ 0.2990973 ,  0.46439536],
       [ 0.54982541,  0.50677575],
       [ 0.30023192,  0.75728772],
       [ 0.38108782,  0.57072784],
       [ 0.48394068,  0.60172994],
       [ 0.62019841,

# Load data
Please use the cell below to discuss your dataset choice and why it is appropriate (or not) for this algorithm.

In [1]:
# Load data

# Normalize data if necessary 

# Split the data into train and test

# t-SNE demonstration 
Demonstrate your t-SNE implementation.

Add plots and figures. The code below is just to help you get started, and should not be your final submission.

Please use the cell below to describe your results and tests.

Describe the difference between your implementation and the sklearn implementation. Hint: you can look at the documentation.

In [ ]:
# Run your custom t-SNE implementation
custom_tsne = CustomTSNE(n_components=2, perplexity=N/10)
custom_Y = custom_tsne.fit_transform(X_train)

# Run sklearn t-SNE
sk_tsne = TSNE(n_components=2, init='random', perplexity=N/10)
sk_Y = sk_tsne.fit_transform(X_train)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

plt.figure()
plt.scatter(sk_Y[:, 0], sk_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with sklearn t-SNE')
plt.show()

# t-SNE extension - mapping new samples
Demonstrate your t-SNE transformation procedure.

Add plots and figures.

Please use the cell below t describe your suggested approach in detail. Use formal notations where appropriate.
Describe and discuss your results.

In [ ]:
# Transform new data
custom_Y_new = custom_tsne.transform(X_train,custom_Y,X_test)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.scatter(custom_Y_new[:, 0], custom_Y_new[:, 1], marker = '*', s=50, linewidths=0.5, edgecolors='k', c=label_test.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

# Use of generative AI
Please use the cell below to describe your use of generative AI in this assignment. 

I asked chatgpt how to calculate efficiently Y_diff